In [ ]:
import os
import numpy as np
from eolearn.core import EOPatch, FeatureType, OverwritePermission
from fs_s3fs import S3FS
from matplotlib import pyplot as plt
from sentinelhub import SHConfig
from sg_utils.processing import multiprocess

In [ ]:
config = SHConfig()
config.sh_client_id = ''
config.sh_client_secret = ''
config.instance_id = ''
config.aws_access_key_id = ''
config.aws_secret_access_key = ''

In [ ]:
filesystem = S3FS(bucket_name='',
                  aws_access_key_id=config.aws_access_key_id,
                  aws_secret_access_key=config.aws_secret_access_key)

In [ ]:
DIR_DEIMOS = ''

In [ ]:
def _clms_eops(eop, threshold=95, band=0):

    clms = []
    for i, ts in enumerate(eop.timestamp):
        gain = float(eop.meta_info['metadata'][ts]['MS4']['PHYSICAL_INFO'][f'PHYSICAL_GAIN_{band+1}'])
        bias = float(eop.meta_info['metadata'][ts]['MS4']['PHYSICAL_INFO'][f'PHYSICAL_BIAS_{band+1}'])
        clms.append(((eop.data['BANDS-DEIMOS'][i, ..., band]*gain + bias) > threshold))
    return np.array(clms)

In [ ]:
def add_clm(eop_path):
    try:
        deim_eop = EOPatch.load(eop_path, filesystem=filesystem, lazy_loading=True)
        deim_eop.mask['CLM'] = np.expand_dims(_clms_eops(deim_eop), -1)
        deim_eop.save(path=eop_path, filesystem=filesystem, features=[(FeatureType.MASK, 'CLM')],
                      overwrite_permission=OverwritePermission.OVERWRITE_FEATURES)
        return True, eop_path
    except Exception:
        return False, eop_path

In [ ]:
eops_paths = [os.path.join(DIR_DEIMOS, x) for x in filesystem.listdir(DIR_DEIMOS)]

In [ ]:
results = multiprocess(add_clm, eops_paths, max_workers=4)

In [ ]:
deim_eop = EOPatch.load(eops_paths[850], filesystem=filesystem, lazy_loading=True)

In [ ]:
fig, ax = plt.subplots(2, figsize=(15, 15))
tidx = 0
ax[0].imshow(deim_eop.data['BANDS-DEIMOS'][tidx][..., [2, 1, 0]].squeeze()*(1/10000))
ax[1].imshow(deim_eop.mask['CLM'][tidx].squeeze())
ax[1].set_title(f"CCOV: {deim_eop.mask['CLM'][tidx].mean()}")